## Popularity Based Recommendation System

In [1]:
## importing all necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
##read text file.
data = pd.read_table('kaggle_visible_evaluation_triplets.txt', header = None)
## reanming column names.
data.columns = ['user_id', 'song_id', 'freq'] 

## reading text file
data_2 = pd.read_table('unique_tracks.txt', sep = '<SEP>',header = None)
## changing column names.
data_2.columns = ['track_id', 'song_id', 'artist_name','release'] 

## reading text file.
data_3 = pd.read_csv('taste_profile_song_to_tracks.txt',sep = '<\t>',header = None)
## renaming column names.
data_3.columns = ['song_to_tracks'] 
data_3['Songs'],data_3['tracks'] = data_3['song_to_tracks'].str.split('\t',1).str
del data_3['song_to_tracks']

## reading text file.
data_4 = pd.read_table('kaggle_songs.txt',header = None)
## renaming column name
data_4.columns = ['songs'] 
data_4['Songs'],data_4['id'] = data_4['songs'].str.split(' ',1).str
del data_4['songs']

## reading text file.
data_5 = pd.read_table('kaggle_users.txt', sep = '<SEP>',header = None)
## renaming column names.
data_5.columns = ['user_id'] 


data_6 = pd.merge(data,data_5,on = 'user_id')
### merging two dataframes(data_3 and data_4)

data_7 = pd.merge(data_3,data_4,on = 'Songs')

dataframe = pd.merge(data_6,data_2,how ='left',on ='song_id')


In [3]:
dataframe.head()

,user_id,song_id,freq,track_id,artist_name,release
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,TRAEHHJ12903CF492F,Dwight Yoakam,You're The One
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,TRLGMFJ128F4217DBE,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,TRTNDNE128F1486812,Cartola,Tive Sim
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,TRASTUE128F930D488,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,TRFPLWO128F1486B9E,Miguel Calo,El Cuatrero


In [4]:
dataframe['song'] = dataframe['release'].map(str) + " - " + dataframe['artist_name']

In [5]:
dataframe.head()

,user_id,song_id,freq,track_id,artist_name,release,song
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,TRAEHHJ12903CF492F,Dwight Yoakam,You're The One,You're The One - Dwight Yoakam
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,TRLGMFJ128F4217DBE,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,TRTNDNE128F1486812,Cartola,Tive Sim,Tive Sim - Cartola
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,TRASTUE128F930D488,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,TRFPLWO128F1486B9E,Miguel Calo,El Cuatrero,El Cuatrero - Miguel Calo


In [6]:
song_df_grouped = dataframe.groupby(['song']).agg({'freq': 'count'}).reset_index()
song_df_grouped.sort_values('freq',ascending = 0)

,song,freq
116533,Sehr kosmisch - Harmonia,5043
147890,Undo - BjÃ¶rk,4483
160451,You're The One - Dwight Yoakam,4136
148622,Use Somebody - Kings Of Leon,3822
33519,Dog Days Are Over (Radio Edit) - Florence + Th...,3780
...,...,...
77970,Life Is Good - LFO feat. M.O.P.,1
77969,Life Is Good (Album Version) - Los Lobos,1
77964,Life Is But A Dream - The Harptones,1
77962,Life Is Beautiful (Listen Album Version) - Mic...,1


In [7]:
users = dataframe['user_id'].unique()
len(users)

110000

In [8]:
items = dataframe['song'].unique()
len(items)

162110

In [9]:
def create_popular_recommendations(data,user_id,item_id):
    # The items are grouped by item_id aggregated with the count of the users and the index is reseted.
    dataframe_grouped = dataframe.groupby([item_id]).agg({user_id: 'count'}).reset_index()
    # The column named user_id is replaced by the name score.
    dataframe_grouped.rename(columns = {'user_id': 'score'}, inplace = True)
    # The training data is sorted according to the score in descending order and by item_id in ascending order.
    dataframe_sort = dataframe_grouped.sort_values(['score', item_id], ascending = [0,1])
    # The new column named Rank is created by score sorted in ascending order.
    dataframe_sort['Rank'] = dataframe_sort['score'].rank(ascending = 0, method = 'first')
    #returning the most_popualar recommendations
    return dataframe_sort.head(15)
    
    
def recommend(user_id,popularity_recommendataions):

    # Init the user_recommendataion var by popularity_recommendataions since the recommendations has been saved into this column.
    user_recommendataion = popularity_recommendataions

    # Get the user_id
    user_recommendataion['user_id'] = user_id

    # Set the columns
    cols = user_recommendataion.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    user_recommendataion = user_recommendataion[cols]

    return user_recommendataion


In [10]:
#Storing the most popular recommendations
popular = create_popular_recommendations(dataframe,'user_id','song')

In [12]:
#Recommending to user
recommend(users[50],popular)

,user_id,song,score,Rank
116533,9de45f3b754cdc54410f1b4529648bdff9c68fca,Sehr kosmisch - Harmonia,5043,1.0
147890,9de45f3b754cdc54410f1b4529648bdff9c68fca,Undo - BjÃ¶rk,4483,2.0
160451,9de45f3b754cdc54410f1b4529648bdff9c68fca,You're The One - Dwight Yoakam,4136,3.0
148622,9de45f3b754cdc54410f1b4529648bdff9c68fca,Use Somebody - Kings Of Leon,3822,4.0
33519,9de45f3b754cdc54410f1b4529648bdff9c68fca,Dog Days Are Over (Radio Edit) - Florence + Th...,3780,5.0
110768,9de45f3b754cdc54410f1b4529648bdff9c68fca,Revelry - Kings Of Leon,3672,6.0
116282,9de45f3b754cdc54410f1b4529648bdff9c68fca,Secrets - OneRepublic,3430,7.0
58017,9de45f3b754cdc54410f1b4529648bdff9c68fca,Horn Concerto No. 4 in E flat K495: II. Romanc...,3272,8.0
123169,9de45f3b754cdc54410f1b4529648bdff9c68fca,Somebody To Love - Justin Bieber,3180,9.0
56260,9de45f3b754cdc54410f1b4529648bdff9c68fca,Hey_ Soul Sister - Train,2791,10.0


In [ ]:
user_data = self.train_data[self.train_data[self.user_id] == user]
user_items = list(user_data[self.item_id].unique())